# Centralized Solution of Multi-Regional Transmission Expansion Horizontal Investment Coordination Problem

### This notebook contains codes for the three zone test case (consisting of the IEEE 14 nodes, IEEE 30 nodes, and a 5 node test system for each of the zones) under the assumption that a single entity solves the problem centrally

#### The first code block loads the different package dependencies to create the virtual environment in Julia

In [179]:
import Pkg
Pkg.add("HiGHS")
Pkg.add("Gurobi")
Pkg.add("GLPK")
Pkg.add("XLSX")
Pkg.add("SCIP")
Pkg.add("Ipopt")
Pkg.add("Cbc")
using Pkg
using JuMP
using GLPK
using XLSX
using SCIP
using Gurobi
using Ipopt
using CSV
using Cbc
using DataFrames

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/

#### The next code block reads the different files corresponding to shared candidate lines, internal candidate lines, shared existing lines, internal existing lines, generators, and load

In [221]:
shared_cand = DataFrame(XLSX.readtable("../Input_Data/CandLine.csv", "Taul1", header=true)...) #Dataframe of shared candidate lines
int_cand =  DataFrame(XLSX.readtable("../Input_Data/CandLineInt.csv", "Taul1", header=true)...) #Dataframe of internal candidate lines
shared_ex =  DataFrame(XLSX.readtable("../Input_Data/SharedEline.csv", "Taul1", header=true)...) #Dataframe of shared existing lines
int_ex = DataFrame(XLSX.readtable("../Input_Data/Tran.csv", "Taul1", header=true)...) #Dataframe of internal existing lines
gen =  DataFrame(XLSX.readtable("../Input_Data/Gen.csv" , "Taul1", header=true)...) #Dataframe of generators
load =  DataFrame(XLSX.readtable("../Input_Data/Load.csv", "Taul1", header=true)...) #Dataframe of loads
scen_prob = DataFrame(CSV.File("../Input_Data/Scenario_Probability.csv", header=true)) #Dataframe of scenario probabilities
zone_summary = DataFrame(CSV.File("../Input_Data/Zone_Summary.csv", header=true)) #Dataframe of scenario probabilities

,Zone,Nodes_Total
,Int64,Int64
1,1,14
2,2,30
3,3,5


In [222]:
shared_cand

,Sernum,tNodeID1,nodeZone1,tNodeID2,nodeZone2,reacT,ptMax,lifeTime,interestRate
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1,1,1,2,3,0.05917,100,50,0.05
2,2,5,3,2,1,0.22304,100,57,0.045
3,3,10,1,4,3,0.19797,100,29,0.03
4,4,19,2,14,1,0.0575,100,35,0.1
5,5,23,2,4,3,0.1652,100,50,0.05
6,6,2,3,15,2,0.1737,100,57,0.045


In [223]:
gen

,zoneNum,gNodeID,C2,C1,C0,PgMax,PgMin
,Any,Any,Any,Any,Any,Any,Any
1,1,1,0.0430293,20,0,332.4,0
2,1,2,0.25,20,0,140,0
3,2,1,0.02,2,0,80,0
4,2,2,0.0175,1.75,0,80,0
5,2,13,0.0625,1,0,40,0
6,2,22,0.00834,3.25,0,50,0
7,2,23,0.025,3,0,30,0
8,2,27,0.025,3,0,55,0
9,3,1,0.0430293,2,0,332.4,0


#### The following code-section constructs several DataFrames for each zone or region, for loads within a particular zone, generators within a particular zone, shared candidate lines originating from or terminating within a particular zone, internal candidate lines within a particular zone, shared existing lines originating from or terminating within a particular zone, internal existing lines within a particular zone, marginal costs of generators within a particular zone, and two binary vectors for each zone, one for checking which all shared candidate lines belong to a particular zone and the other for checking which all shared existing lines belong to a particular zone.

In [224]:
l(i,s) = load[load.zoneNum .== i, [1,2,s+2]] # load within zone i
g(i) = gen[gen.zoneNum .== i, :]   # generators within zone i
shared_c(i) = vcat(shared_cand[shared_cand.nodeZone1 .== i,:] , shared_cand[shared_cand.nodeZone2 .== i, :]) #shared candidate lines within zone i
int_c(i) = int_cand[int_cand.zoneNum .== i, :]   # number of internal candidate lines within zone i
shared_e(i) = vcat(shared_ex[shared_ex.nodeZone1 .== i,:] , shared_ex[shared_ex.nodeZone2 .== i, :]) #shared existing lines within zone i
int_e(i) =int_ex[int_ex.zoneNum .== i, :]       # internal existing lines within zone i
MC(i) = (g(i).C2 .* (g(i).PgMax .^ 2) .+ g(i).C1 .* g(i).PgMax .- g(i).C2 .*(g(i).PgMin .^ 2) .- g(i).C1 .* g(i).PgMin) ./ (g(i).PgMax .- g(i).PgMin) #Marginal cost of generators within zone i
bin_c(i) = (shared_cand.nodeZone1 .== i) + (shared_cand.nodeZone2 .== i) # A binary vector through which we can check if the shared candidate lines belong to zone i
bin_e(i) = (shared_ex.nodeZone1 .== i) + (shared_ex.nodeZone2 .== i) # A binary vector through which we can check if the shared existing lines belong to zone i
scen_weight(s) = scen_prob.scen_weight[scen_prob.scenario .== s,:]

scen_weight (generic function with 1 method)

In [225]:
typeof(scen_weight(1)[1])


Int64

In [226]:
scen_weight(1)[1]

1

In [227]:
typeof(g(1))

DataFrame

In [228]:
l(3,1)

,zoneNum,lNodeID,P_load1
,Any,Any,Any
1,3,2,-20
2,3,3,-45
3,3,4,-40
4,3,5,-60


In [229]:
MC(1)

2-element Array{Float64,1}:
 34.30293932
 55.0

#### Constructing the optimization model with a specified solver

In [230]:
g(2)

,zoneNum,gNodeID,C2,C1,C0,PgMax,PgMin
,Any,Any,Any,Any,Any,Any,Any
1,2,1,0.02,2,0,80,0
2,2,2,0.0175,1.75,0,80,0
3,2,13,0.0625,1,0,40,0
4,2,22,0.00834,3.25,0,50,0
5,2,23,0.025,3,0,30,0
6,2,27,0.025,3,0,55,0


In [231]:
Mod3 = Model(Gurobi.Optimizer)

Academic license - for non-commercial use only - expires 2023-09-07


A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Gurobi

In [232]:
zone_number = 3;

In [233]:
@variable(Mod3,0 <= gen_var_1[1:14,1:nrow(scen_prob)]) # generation at node 1
@variable(Mod3,0 <= gen_var_2[1:30,1:nrow(scen_prob)]) # generation at node 2
@variable(Mod3,0 <= gen_var_3[1:5,1:nrow(scen_prob)]) # generation at node 3

@variable(Mod3,shared_line_decision_var[1:nrow(shared_cand)], Bin) #Decision variable for shared candidate lines

@variable(Mod3,int_line_decision_var_1[1:nrow(int_c(1))], Bin) #Decision variable for internal candidate lines of zone 1 (Binary)
@variable(Mod3,int_line_decision_var_2[1:nrow(int_c(2))], Bin) #Decision variable for internal candidate lines of zone 2 (Binary)
@variable(Mod3,int_line_decision_var_3[1:nrow(int_c(3))], Bin) #Decision variable for internal candidate lines of zone 3 (Binary)

@variable(Mod3,shared_cand_flow[1:nrow(shared_cand),1:nrow(scen_prob)])  #Power flowing on shared candidate lines 

@variable(Mod3, int_cand_flow_1[1:nrow(int_c(1)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 1 
@variable(Mod3, int_cand_flow_2[1:nrow(int_c(2)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 2
@variable(Mod3, int_cand_flow_3[1:nrow(int_c(3)),1:nrow(scen_prob)]) #Power flowing on internal candidate lines of zone 3

@variable(Mod3, 0 <= node_voltage_phase_angle[i=1:nrow(zone_summary),j=1:zone_summary.Nodes_Total[i], 1:nrow(scen_prob)]<= 2*pi)

#@variable(Mod3,0 <= shared_cand_angle[1:nrow(shared_cand),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for shared candidate lines

#@variable(Mod3,0 <= int_cand_angle_1[1:nrow(int_c(1)),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 1 
#@variable(Mod3,0 <= int_cand_angle_2[1:nrow(int_c(2)),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 2
#@variable(Mod3,0 <= int_cand_angle_3[1:nrow(int_c(3)),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for internal candidate lines of zone 3 

@variable(Mod3, shared_ex_flow[1:nrow(shared_ex),1:nrow(scen_prob)])  #Power flowing on shared existing shared lines

@variable(Mod3, int_ex_flow_1[1:nrow(int_e(1)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 1 
@variable(Mod3, int_ex_flow_2[1:nrow(int_e(2)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 2
@variable(Mod3, int_ex_flow_3[1:nrow(int_e(3)),1:nrow(scen_prob)])  #Power flowing on internal existing lines of zone 3

#@variable(Mod3,0 <= shared_ex_angle[1:nrow(shared_ex),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle for existing shared lines

#@variable(Mod3,0 <= int_ex_angle_1[1:nrow(int_e(1)),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 1
#@variable(Mod3,0 <= int_ex_angle_2[1:nrow(int_e(2)),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 2
#@variable(Mod3,0 <= int_ex_angle_3[1:nrow(int_e(3)),1:nrow(scen_prob),1:2]<= 2*pi) #Phase angle decision for existing internal lines of zone 3


7×1 Array{VariableRef,2}:
 int_ex_flow_3[1,1]
 int_ex_flow_3[2,1]
 int_ex_flow_3[3,1]
 int_ex_flow_3[4,1]
 int_ex_flow_3[5,1]
 int_ex_flow_3[6,1]
 int_ex_flow_3[7,1]

In [234]:
@expression(Mod3,total_cost,0)

0

In [235]:
@expression(Mod3, gen_costNS1[n=1:14,s=1:nrow(scen_prob)], (gen_var_1[n,s] .* sum((g(1).gNodeID .== n) .* MC(1))))
@expression(Mod3, gen_cost_prob_weightedS1[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS1[n,s] for n in 1:14))

1-element Array{GenericAffExpr{Float64,VariableRef},1}:
 34.30293932 gen_var_1[1,1] + 55 gen_var_1[2,1]

In [236]:
@expression(Mod3, gen_cost_expected1, sum(gen_cost_prob_weightedS1[s] for s in 1:nrow(scen_prob)))
Mod3[:total_cost] += gen_cost_expected1

34.30293932 gen_var_1[1,1] + 55 gen_var_1[2,1]

In [237]:
@expression(Mod3, gen_costNS2[n=1:30,s=1:nrow(scen_prob)], (gen_var_2[n,s] .* sum((g(2).gNodeID .== n) .* MC(2))))

30×1 Array{GenericAffExpr{Float64,VariableRef},2}:
 3.6 gen_var_2[1,1]
 3.15 gen_var_2[2,1]
 0
 0
 0
 0
 0
 0
 0
 0
 0
 0
 3.5 gen_var_2[13,1]
 ⋮
 0
 0
 0
 3.667 gen_var_2[22,1]
 3.75 gen_var_2[23,1]
 0
 0
 0
 4.375 gen_var_2[27,1]
 0
 0
 0

In [238]:
@expression(Mod3, gen_cost_prob_weightedS2[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS2[n,s] for n in 1:30))

1-element Array{GenericAffExpr{Float64,VariableRef},1}:
 3.6 gen_var_2[1,1] + 3.15 gen_var_2[2,1] + 3.5 gen_var_2[13,1] + 3.667 gen_var_2[22,1] + 3.75 gen_var_2[23,1] + 4.375 gen_var_2[27,1]

In [239]:
@expression(Mod3, gen_cost_expected2, sum(gen_cost_prob_weightedS2[s] for s in 1:nrow(scen_prob)))

3.6 gen_var_2[1,1] + 3.15 gen_var_2[2,1] + 3.5 gen_var_2[13,1] + 3.667 gen_var_2[22,1] + 3.75 gen_var_2[23,1] + 4.375 gen_var_2[27,1]

In [240]:
Mod3[:total_cost]+=gen_cost_expected2

34.30293932 gen_var_1[1,1] + 55 gen_var_1[2,1] + 3.6 gen_var_2[1,1] + 3.15 gen_var_2[2,1] + 3.5 gen_var_2[13,1] + 3.667 gen_var_2[22,1] + 3.75 gen_var_2[23,1] + 4.375 gen_var_2[27,1]

In [241]:
@expression(Mod3, gen_costNS3[n=1:5,s=1:nrow(scen_prob)], (gen_var_3[n,s] .* sum((g(3).gNodeID .== n) .* MC(3))))
@expression(Mod3, gen_cost_prob_weightedS3[s=1:nrow(scen_prob)], (scen_weight(s)[1]).*sum(gen_costNS3[n,s] for n in 1:5))

1-element Array{GenericAffExpr{Float64,VariableRef},1}:
 16.30293932 gen_var_3[1,1] + 37 gen_var_3[2,1]

In [242]:
@expression(Mod3, gen_cost_expected3, sum(gen_cost_prob_weightedS3[s] for s in 1:nrow(scen_prob)))

16.30293932 gen_var_3[1,1] + 37 gen_var_3[2,1]

In [243]:
Mod3[:total_cost]+=gen_cost_expected3

34.30293932 gen_var_1[1,1] + 55 gen_var_1[2,1] + 3.6 gen_var_2[1,1] + 3.15 gen_var_2[2,1] + 3.5 gen_var_2[13,1] + 3.667 gen_var_2[22,1] + 3.75 gen_var_2[23,1] + 4.375 gen_var_2[27,1] + 16.30293932 gen_var_3[1,1] + 37 gen_var_3[2,1]

In [244]:
@expression(Mod3, investment_cost , (sum(shared_line_decision_var[c] .* shared_cand.costPerCap[c] .* shared_cand.interestRate[c] 
            .*((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c]) ./ (((1 + shared_cand.interestRate[c]) .^ shared_cand.lifeTime[c])-1) for c in 1:nrow(shared_cand))
            .+ sum(int_line_decision_var_1[c] .* int_c(1).costPerCap[c] .* int_c(1).interestRate[c] 
            .*((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c]) ./ (((1 + int_c(1).interestRate[c]) .^ int_c(1).lifeTime[c])-1) for c in 1:nrow(int_c(1)))
            .+ sum(int_line_decision_var_2[c] .* int_c(2).costPerCap[c] .* int_c(2).interestRate[c] 
            .*((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c]) ./ (((1 + int_c(2).interestRate[c]) .^ int_c(2).lifeTime[c])-1) for c in 1:nrow(int_c(2)))
            .+ sum(int_line_decision_var_3[c] .* int_c(3).costPerCap[c] .* int_c(3).interestRate[c] 
            .*((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c]) ./ (((1 + int_c(3).interestRate[c]) .^ int_c(3).lifeTime[c])-1) for c in 1:nrow(int_c(3)))))
Mod3[:total_cost]+=investment_cost

34.30293932 gen_var_1[1,1] + 55 gen_var_1[2,1] + 3.6 gen_var_2[1,1] + 3.15 gen_var_2[2,1] + 3.5 gen_var_2[13,1] + 3.667 gen_var_2[22,1] + 3.75 gen_var_2[23,1] + 4.375 gen_var_2[27,1] + 16.30293932 gen_var_3[1,1] + 37 gen_var_3[2,1] + 0.2410176361372405 shared_line_decision_var[1] + 0.19594024895900847 shared_line_decision_var[2] + 0.44297470440637654 shared_line_decision_var[3] + 0.6532451422553538 shared_line_decision_var[4] + 0.1533748593600621 shared_line_decision_var[5] + 0.34289543567826486 shared_line_decision_var[6] + 0.1533748593600621 int_line_decision_var_1[1] + 0.34289543567826486 int_line_decision_var_1[2] + 0.2519729832343877 int_line_decision_var_1[3] + 0.21553427385490934 int_line_decision_var_1[4] + 0.44297470440637654 int_line_decision_var_1[5] + 6.532451422553539 int_line_decision_var_1[6] + 1.5337485936006214 int_line_decision_var_1[7] + 3.673879667981409 int_line_decision_var_1[8] + 2.355399625886668 int_line_decision_var_1[9] + 2.400268049747854 int_line_decision_v

In [245]:
#The first zone
for n in 1:14
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n,s] .+ sum(l(1,s)[:,3].* (l(1,s).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(1) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(1) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(1) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(1) .* shared_ex_flow[:,s]) .+
        sum((int_c(1).tNodeID1 .== n) .* int_cand_flow_1[:,s]) .- sum((int_c(1).tNodeID2 .== n) .* int_cand_flow_1[:,s]) .+
        sum((int_e(1).tNodeID1 .== n) .* int_ex_flow_1[:,s]) .- sum((int_e(1).tNodeID2 .== n) .* int_ex_flow_1[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod3, sum(g(1).gNodeID .== n) .* gen_var_1[n,s] .<= sum((g(1).gNodeID .== n) .* g(1).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod3, sum((g(1).gNodeID .== n) .* g(1).PgMin) .<= sum(g(1).gNodeID .== n) .* gen_var_1[n,s])
    end
end

In [246]:
#The second zone
for n in 1:30
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n,s] .+ sum(l(2,s)[:,3] .* (l(2,s).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(2) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(2) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(2) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(2) .* shared_ex_flow[:,s]) .+
        sum((int_c(2).tNodeID1 .== n) .* int_cand_flow_2[:,s]) .- sum((int_c(2).tNodeID2 .== n) .* int_cand_flow_2[:,s]) .+
        sum((int_e(2).tNodeID1 .== n) .* int_ex_flow_2[:,s]) .- sum((int_e(2).tNodeID2 .== n) .* int_ex_flow_2[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod3, sum(g(2).gNodeID .== n) .* gen_var_2[n,s] .<= sum((g(2).gNodeID .== n) .* g(2).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod3, sum((g(2).gNodeID .== n) .* g(2).PgMin) .<= sum(g(2).gNodeID .== n) .* gen_var_2[n,s])
    end
end

In [247]:
#The third zone
for n in 1:5
    for s in 1:nrow(scen_prob)
        # Power balance constraint for each node
        @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n,s] .+ sum(l(3,s)[:,3] .* (l(3,s).lNodeID .== n)) .==
        sum((shared_cand.tNodeID1 .== n) .*bin_c(3) .* shared_cand_flow[:,s]) .- sum((shared_cand.tNodeID2 .== n) .* bin_c(3) .* shared_cand_flow[:,s]) .+
        sum((shared_ex.tNodeID1 .== n) .* bin_e(3) .* shared_ex_flow[:,s]) .- sum((shared_ex.tNodeID2 .== n) .* bin_e(3) .* shared_ex_flow[:,s]) .+
        sum((int_c(3).tNodeID1 .== n) .* int_cand_flow_3[:,s]) .- sum((int_c(3).tNodeID2 .== n) .* int_cand_flow_3[:,s]) .+
        sum((int_e(3).tNodeID1 .== n) .* int_ex_flow_3[:,s]) .- sum((int_e(3).tNodeID2 .== n) .* int_ex_flow_3[:,s]))
        #Lower limit for generation of each node
        @constraint(Mod3, sum(g(3).gNodeID .== n) .* gen_var_3[n,s] .<= sum((g(3).gNodeID .== n) .* g(3).PgMax))
        #Upper limit for generation of each node
        @constraint(Mod3, sum((g(3).gNodeID .== n) .* g(3).PgMin) .<= sum(g(3).gNodeID .== n) .* gen_var_3[n,s])
    end
end

In [248]:
#Shared candidate lines
M = 100
for c in 1:nrow(shared_cand)
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- shared_line_decision_var[c]) .<= shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])))
        @constraint(Mod3, shared_cand_flow[c,s] .- ((1 ./ shared_cand.reacT[c]) .* (node_voltage_phase_angle[shared_cand.nodeZone1[c],shared_cand.tNodeID1[c],s] .- node_voltage_phase_angle[shared_cand.nodeZone2[c],shared_cand.tNodeID2[c],s])) .<= M .* (1 .- shared_line_decision_var[c]))
        #limiting the upper bound of power flow flowing within candidate shared lines
        @constraint(Mod3, shared_cand_flow[c,s] .<= shared_line_decision_var[c] .*shared_cand.ptMax[c])
        #Limiting the lower bound of power flowing within the candidate shared lines
        @constraint(Mod3, -shared_cand.ptMax[c] .* shared_line_decision_var[c] .<= shared_cand_flow[c,s])
    end
end

In [249]:
#print(shared_cand.nodeZone2[3])
print(shared_cand.tNodeID2[3])
        

4

In [250]:
#Shared existing lines
for h in 1:nrow(shared_ex)
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, shared_ex_flow[h,s] .== (1 ./ shared_ex.reacT[h]) .* (node_voltage_phase_angle[shared_ex.nodeZone1[h],shared_ex.tNodeID1[h],s] .- node_voltage_phase_angle[shared_ex.nodeZone2[h],shared_ex.tNodeID2[h],s]))
        @constraint(Mod3, shared_ex_flow[h,s] .<= shared_ex. ptMax[h])
        @constraint(Mod3, -shared_ex.ptMax[h] .<= shared_ex_flow[h,s])
    end
end

In [251]:
#Zone 1 internal candidate lines
for c in 1:nrow(int_c(1))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- int_line_decision_var_1[c]) .<= int_cand_flow_1[c,s] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s])))
        @constraint(Mod3, int_cand_flow_1[c,s] .- ((1 ./ int_c(1).reacT[c]) .* (node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(1).zoneNum[c],int_c(1).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_1[c]))
        @constraint(Mod3, int_cand_flow_1[c,s] .<= int_line_decision_var_1[c] .* int_c(1).ptMax[c])
        @constraint(Mod3, -int_c(1).ptMax[c] .*int_line_decision_var_1[c] .<= int_cand_flow_1[c,s])
    end
end

In [252]:
#Zone 2 internal candidate lines
for c in 1:nrow(int_c(2))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- int_line_decision_var_2[c]) .<= int_cand_flow_2[c,s] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s])))
        @constraint(Mod3, int_cand_flow_2[c,s] .- ((1 ./ int_c(2).reacT[c]) .* (node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(2).zoneNum[c],int_c(2).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_2[c]))
        @constraint(Mod3, int_cand_flow_2[c,s] .<= int_line_decision_var_2[c] .* int_c(2).ptMax[c])
        @constraint(Mod3, -int_c(2).ptMax[c] .*int_line_decision_var_2[c] .<= int_cand_flow_2[c,s])
    end
end

In [253]:
#Zone 3 internal candidate lines
for c in 1:nrow(int_c(3))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3,-M .* (1 .- int_line_decision_var_3[c]) .<= int_cand_flow_3[c,s] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s])))
        @constraint(Mod3, int_cand_flow_3[c,s] .- ((1 ./ int_c(3).reacT[c]) .* (node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID1[c],s] .- node_voltage_phase_angle[int_c(3).zoneNum[c],int_c(3).tNodeID2[c],s])) .<= M .* (1 .- int_line_decision_var_3[c]))
        @constraint(Mod3, int_cand_flow_3[c,s] .<= int_line_decision_var_3[c] .* int_c(3).ptMax[c])
        @constraint(Mod3, -int_c(3).ptMax[c] .*int_line_decision_var_3[c] .<= int_cand_flow_3[c,s])
    end
end

In [254]:
#Zone 1 internal existing lines
for h in 1:nrow(int_e(1))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, int_ex_flow_1[h,s] .== (1 ./ int_e(1).reacT[h]) .* (node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(1).zoneNum[h],int_e(1).tNodeID2[h],s]))
        @constraint(Mod3, int_ex_flow_1[h,s] .<= int_e(1).ptMax[h])
        @constraint(Mod3, -int_e(1).ptMax[h] .<= int_ex_flow_1[h,s])
    end
end

In [255]:
#Zone 2 internal existing lines
for h in 1:nrow(int_e(2))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, int_ex_flow_2[h,s] .== (1 ./ int_e(2).reacT[h]) .* (node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(2).zoneNum[h],int_e(2).tNodeID2[h],s]))
        @constraint(Mod3, int_ex_flow_2[h,s] .<= int_e(2).ptMax[h])
        @constraint(Mod3, -int_e(2).ptMax[h] .<= int_ex_flow_2[h,s])
    end
end

In [256]:
#Zone 3 internal existing lines
for h in 1:nrow(int_e(3))
    for s in 1:nrow(scen_prob)
        @constraint(Mod3, int_ex_flow_3[h,s] .== (1 ./ int_e(3).reacT[h]) .* (node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID1[h],s] .- node_voltage_phase_angle[int_e(3).zoneNum[h],int_e(3).tNodeID2[h],s]))
        @constraint(Mod3, int_ex_flow_3[h,s] .<= int_e(3).ptMax[h])
        @constraint(Mod3, -int_e(3).ptMax[h] .<= int_ex_flow_3[h,s])
    end
end

In [257]:
@objective(Mod3, Min, Mod3[:total_cost])
optimize!(Mod3)

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 519 rows, 246 columns and 1082 nonzeros
Model fingerprint: 0xf3a49630
Variable types: 210 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e-01, 6e+01]
  Bounds range     [6e+00, 6e+00]
  RHS range        [2e+00, 3e+02]
Presolve removed 318 rows and 108 columns
Presolve time: 0.01s
Presolved: 201 rows, 138 columns, 751 nonzeros
Variable types: 103 continuous, 35 integer (35 binary)

Root relaxation: objective 9.837305e+03, 200 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 9837.30470    0   34          - 9837.30470      -     -    0s
     0     0 infeasible    0               - infeasible      -     -    0s

Cutting planes:
  Gom

In [161]:
if termination_status(Mod3) == MOI.OPTIMAL
    shared_line_decision = value.(shared_line_decision_var)
    shared_cand_power = value.(shared_cand_flow)
    #shared_cand_phase_angle = value.(shared_cand_angle)
    shared_ex_power = value.(shared_ex_flow)
    #shared_ex_phase_angle = value.(shared_ex_angle)
    int_line_decision_1 = value.(int_line_decision_var_1)
    int_line_decision_2 = value.(int_line_decision_var_2)
    int_line_decision_3 = value.(int_line_decision_var_3)
    int_line_flow_1 = value.(int_cand_flow_1)
    int_line_flow_2 = value.(int_cand_flow_2)
    int_line_flow_3 = value.(int_cand_flow_3)
    generation_1 = value.(gen_var_1)
    generation_2 = value.(gen_var_2)
    generation_3 = value.(gen_var_3)
    obj_value = objective_value(Mod3)
end

2237.7825988633294

In [162]:
shared_line_decision

6-element Array{Float64,1}:
 -0.0
 -0.0
  1.0
  1.0
  1.0
 -0.0

In [163]:
shared_cand_power

6×4 Array{Float64,2}:
   0.0        0.0        0.0       0.0
   0.0        0.0        0.0       0.0
  31.7381    31.7381    30.0621   31.7381
 100.0      100.0      100.0      41.4789
   5.90588    3.11727   -2.5387  -38.0338
   0.0        0.0        0.0       0.0

In [164]:
shared_cand_phase_angle

6×4×2 Array{Float64,3}:
[:, :, 1] =
 0.366185  0.366185  6.28319  0.366185
 0.0       0.0       0.0      0.0
 6.28319   6.28319   5.95139  6.28319
 5.75      5.75      6.28319  6.28319
 0.975651  6.28319   0.0      0.0
 6.28319   0.0       6.28319  6.28319

[:, :, 2] =
 6.28319  6.28319  0.366185  6.28319
 6.28319  6.28319  6.28319   6.28319
 0.0      0.0      0.0       0.0
 0.0      0.0      0.533185  3.89815
 0.0      5.76821  0.419394  6.28319
 0.0      6.28319  0.0       0.0

In [165]:
shared_ex_phase_angle

8×4×2 Array{Float64,3}:
[:, :, 1] =
 6.28319  6.28319  6.28319  6.28319
 0.0      0.0      0.0      0.0
 0.0      0.0      0.0      0.0
 6.28319  6.28319  0.0      0.0
 6.28319  6.1149   3.72122  5.69315
 6.28319  6.28319  6.28319  5.4377
 2.47282  0.0      2.47282  6.28319
 0.0      0.0      0.0      0.0

[:, :, 2] =
 8.88178e-16  0.0          8.88178e-16  8.88178e-16
 6.28319      6.28319      6.28319      6.28319
 0.0          0.0          0.0          0.0
 0.0          1.51268      6.28319      6.28319
 2.34581      6.28319      0.0          6.28319
 0.0          8.88178e-16  0.0          0.0
 0.0          6.28319      0.0          6.2699
 6.28319      3.95996      0.558859     6.28319

In [166]:
shared_ex_power

8×4 Array{Float64,2}:
  23.2711   23.2711    23.2711    23.2711
 -19.0862  -19.0862   -19.0862   -19.0862
   0.0       0.0        0.0        0.0
  11.3007    8.58005  -11.3007   -11.3007
  65.7325   -2.80952   62.1239    -9.8504
  31.4159   31.4159    31.4159    27.1885
  22.4801  -57.1199    22.4801     0.120754
 -30.2076  -19.0383    -2.68682  -30.2076

In [167]:
int_line_decision_1

10-element Array{Float64,1}:
 -0.0
  1.0
  1.0
 -0.0
  1.0
  1.0
 -0.0
 -0.0
 -0.0
  1.0

In [168]:
int_line_decision_2

18-element Array{Float64,1}:
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
  1.0
  1.0
 -0.0
 -0.0
  1.0
  1.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0
 -0.0

In [169]:
int_line_decision_3

2-element Array{Float64,1}:
 -0.0
 -0.0

In [170]:
obj_value

2237.7825988633294

In [171]:
generation_1

14×4 Array{Float64,2}:
 135.729  6.22894  70.9789  0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0
   0.0    0.0       0.0     0.0

In [172]:
show(stdout, "text/plain", generation_2)

30×4 Array{Float64,2}:
 28.292   0.0  21.9   0.0
 80.0    80.0  80.0  47.3
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
 40.0    14.6  40.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
 40.908   0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0
  0.0     0.0   0.0   0.0

In [173]:
generation_3

5×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0

In [174]:
int_line_flow_1

10×4 Array{Float64,2}:
   0.0       0.0       0.0        0.0
  35.6351   35.6351   35.6351    35.6351
 -36.1352  -36.1352  -36.1352   -36.1352
   0.0       0.0       0.0        0.0
  87.0525  -25.2522   29.4941  -100.0
 -30.0458  -30.0458  -22.9706   -30.0458
   0.0       0.0       0.0        0.0
   0.0       0.0       0.0        0.0
   0.0       0.0       0.0        0.0
  13.7617   21.2117   24.5619    12.8924

In [175]:
int_line_flow_2

18×4 Array{Float64,2}:
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
 -31.6215  -30.6719  -26.9955  -31.6215
 -15.8399  -10.2117  -13.606   -31.4631
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
 -32.6739  -32.6739  -32.6739  -32.6739
 -47.7305  -48.6315  -48.6315  -48.6315
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0       0.0
   0.0       0.0       0.0      -4.44089e-16
   0.0       0.0       0.0       0.0

In [176]:
int_line_flow_3

2×4 Array{Float64,2}:
 0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0